In [6]:
%matplotlib widget
''' 系统库 '''
import datetime
import time
from random import randint 
from importlib import reload 
# from numba import jit
# @jit(nopython=True)

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

''' 业务库 '''
# import mplfinance as mpf
import akshare as ak

''' 自建库 '''
from qflib import basic
reload(basic)

global engine, conn 
engine = basic.engine()
conn = basic.conn(engine)

trade_date = datetime.datetime.now().strftime('%Y%m%d')
print('date : ', trade_date)
run_static, run_dynamic = False, True
# run_static, run_dynamic = True, True
# run_static, run_dynamic = True, False


date :  20230731


In [7]:
def tran_daily_ak(zero_init, symbol, start_date, end_date) :
    
    # df = ts.pro_bar(ts_code=ts_code, adj='qfq', start_date=start_date, end_date=end_date, ma=[5,10,20,30,60,120,250])
    df = ak.stock_zh_a_hist(symbol=symbol, period="daily", start_date=start_date, end_date=end_date, adjust="qfq")

    print( symbol, '- read: ', df.shape, end = "")
    if len(df) > 0:  
        df['symbol'] = symbol
        df.rename( columns={'日期':'trade_date', '开盘':'open', '收盘':'close', '最高':'high', '最低':'low', 
                            '成交量':'volume','涨跌幅':'pct_change','成交额':'value','换手率':'exg_rate', 
                            }, inplace=True )
        del df['振幅'], df['涨跌额']
        df.drop(index=(df[ df['close']<=0 ].index),inplace=True)    # qfq模式下，股价负数的不再考虑。
        df['pre_close'] = df['close'].shift(1)                      # 用上一条的数据作为今天的pre_close
        df.at[0,'pre_close'] = df.at[0,'close']/(1+df.at[0,'pct_change']/100) # 第一提记录的pre_close无法获得，所以用计算获得。
        df['change'] = df['close'] - df['pre_close']                # 生成 change

        if not zero_init:
            sql = "delete from tran_daily where symbol='" + symbol +"'"
            basic.exec_sql(conn,sql)
        write_no = basic.write_data(conn, df, 'tran_daily', False,'append')
        print( ',  write: ', write_no)

In [8]:
# %%time
# # 历史行情数据
# df = ak.stock_zh_a_hist(symbol="601318", period="daily", start_date="20160101", end_date='20231231', adjust="qfq")
# del df['振幅'], df['涨跌额']
# df['symbol'] = '601835'
# df.rename( columns={'日期':'trade_date', '开盘':'open', '收盘':'close', '最高':'high', '最低':'low', 
#                     '成交量':'volume','涨跌幅':'pct_change','成交额':'value','换手率':'exg_rate', 
#                     }, inplace=True )



In [9]:
# df['pre_close'] = df['close'].shift(1)
# df.at[0,'pre_close'] = df.at[0,'close']/(1+df.at[0,'pct_change']/100)
# df['previous']=df['current'].shift(1)
# stock_zh_a_hist_df.info()
# stock_zh_a_hist_df.info


In [10]:
# df

In [11]:
# df.iloc[0,'pre_close']
# df.at[0,'pre_close'] = df.at[0,'close']/(1+df.at[0,'pct_change']/100)

In [12]:
%%time
"""  动态数据更新：每日成交、每日MoneyFlow  """

""" v2.0 qfq 模式"""

# get ts_list to update  
sql="SELECT * FROM stock_basic"
df_tslist = pd.read_sql_query(sql, conn)
df_tslist_size = len( df_tslist )
print(  df_tslist.shape  )

# 是否清空 tran_daily 
zero_init = 1 
if zero_init == True:    
    sql = "truncate table tran_daily"
    basic.exec_sql(conn,sql)
    start_date, end_date = '20190101', '20231231'
else:
    start_date, end_date = '20190101', '20231231'
    # pass

# for i in range( 3 ) :
for i in range( len(df_tslist) ) :
    print( i, end=': ')
    symbol = df_tslist['symbol'][i]
    # print(symbol, type(symbol))
    tran_daily_ak(zero_init, symbol, start_date, end_date )
    # print(ts_list.index[i]+1,' - ',str(ts_list_size), ts_list['ts_code'][i], end=" ")
    
"""  V1.0 """
# days = 3
# days = int(365 * 3)
# days = 365

# 每日成交数据 -- OK
# sync_tran_daily(conn, days)
# 每日资金流向  --  暂停
# .sync_moneyflow(engine, days)

# 1、国内股市：可显示复权后和复权前信息, 现在用前复权价格
# df = ts.pro_bar(ts_code=sl_list['ts_code'][i], asset='E', adj='qfq', start_date=start_date, end_date='')

(5234, 7)
0: 000001 - read:  (1111, 11),  write:  1111
1: 000002 - read:  (1111, 11),  write:  1111
2: 000004 - read:  (1098, 11),  write:  1098
3: 000005 - read:  (1110, 11),  write:  1110
4: 000006 - read:  (1111, 11),  write:  1111
5: 000007 - read:  (1108, 11),  write:  1108
6: 000008 - read:  (1111, 11),  write:  1111
7: 000009 - read:  (1111, 11),  write:  1111
8: 000010 - read:  (1109, 11),  write:  1109
9: 000011 - read:  (1111, 11),  write:  1111
10: 000012 - read:  (1111, 11),  write:  1111
11: 000014 - read:  (1111, 11),  write:  1111
12: 000016 - read:  (1102, 11),  write:  1102
13: 000017 - read:  (1109, 11),  write:  1109
14: 000019 - read:  (1111, 11),  write:  1111
15: 000020 - read:  (1111, 11),  write:  1111
16: 000021 - read:  (1111, 11),  write:  1111
17: 000023 - read:  (1110, 11),  write:  1110
18: 000025 - read:  (1111, 11),  write:  1111
19: 000026 - read:  (1111, 11),  write:  1111
20: 000027 - read:  (1111, 11),  write:  1111
21: 000028 - read:  (1111, 11),  w

'  V1.0 '

In [13]:

# import cStringIO
 
# output = cStringIO.StringIO()
# # ignore the index
# df_a.to_csv(output, sep='\t',index = False, header = False)
# output.getvalue()
# # jump to start of stream
# output.seek(0)
 
# connection = engine.raw_connection() #engine 是 from sqlalchemy import create_engine
# cursor = connection.cursor()
# # null value become ''
# cursor.copy_from(output,table_name,null='')
# connection.commit()


In [14]:
'''管理层薪酬和持股; 需要高级权限 '''
# df = pro.stk_rewards(ts_code='601318.SH')

'''--- 新股上市清单， 120积分 '''
# df = pro.new_share(start_date='20220101', end_date='20221231')
# df.tail(10)

''' 概念股清单 
    58,TS58,锂电隔膜,ts
    59,TS59,锂电设备,ts
    60,TS60,锂电原料,ts
    61,TS61,锂电制造,ts
'''
# df = pro.concept_detail(id='TS60', fields='ts_code,name') 
# df.to_csv('~/win-desk/lidianyuanliao.txt', index=False, header=False)


' 概念股清单 \n    58,TS58,锂电隔膜,ts\n    59,TS59,锂电设备,ts\n    60,TS60,锂电原料,ts\n    61,TS61,锂电制造,ts\n'

In [15]:
# 获得 bak_basic
def sync_basic(conn, df, table, trade_date):    
    sql = " truncate table "+table
    print( table, basic.exec_sql(conn, sql) )
    basic.write_data(conn, df, table, False, 'append')

# 静态数据更新  -  调整为手工处理
if run_static:
    print( '更新基本信息' )
    df = pro.stock_basic(exchange='', list_status='L')
    sync_basic(engine, df, 'stock_basic', trade_date)
    
    print( "更新公司信息")
    df = pro.stock_company(exchange='', ts_code='')  
    sync_basic(engine, df, 'stock_company', trade_date)
    
    # print( "获得股票关键市场信息； 5000积分以上，否则每分钟最多2次, 每天20次；")
    # df = pro.bak_basic(trade_date=trade_date)
    # sync_basic(engine, df, 'bak_basic', trade_date)

    print( "更新深股通清单")
    df = pro.hs_const(hs_type='SZ')
    sync_basic(engine, df, 'sl_shengutong', trade_date)

    print("更新沪股通清单")
    df = pro.hs_const(hs_type='SH')
    sync_basic(engine, df, 'sl_hugutong', trade_date)

    print("开始改名的股票")
    df = pro.namechange(ts_code='', start_date='2019-01-01', fields='ts_code,name,start_date,end_date,change_reason')
    sync_basic(engine, df, 'namechange', trade_date)

    print("交易日历")
    df = pro.trade_cal(exchange='SSE', start_date='20190101', end_date='20221231')
    sync_basic(engine, df, 'calendar', trade_date)
    
    print("股票概念清单")   # 879+个
    df = pro.concept(src='ts') 
    sync_basic(engine, df, 'concept', trade_date)

# 获得股票日资金流 - OK
def sync_moneyflow(conn, period=1):
    now = datetime.datetime.now()
    # period = 825
    for n in range(-period+1, 1):
        delta = datetime.timedelta(days=n)
        new_day = now + delta
        trade_date = new_day.strftime('%Y%m%d')
        print( trade_date, end="" )  

        # 判断当日数据是否已存在
        sql="SELECT * FROM moneyflow where trade_date='" + trade_date+ "' LIMIT 10"
        df=basic.read_data(conn, sql)
        if len(df) > 0:
            print( '  - existed' )   
        else:    
            # 读取数据
            df = pro.moneyflow(trade_date=trade_date)
            print( '  - read: ', df.shape, end="")
            if len(df) == 0:
                print( '  - no found at Tushare' )
            else:
                # 删除可能的已有数据
                # sql = "delete from moneyflow where trade_date="+trade_date
                # print( 'trade_date :', trade_date )
                # print( 'delete res: ', basic.exec_sql(conn, sql) )
                # 添加数据
                basic.write_data(conn, df, 'moneyflow', False,'append') 
                print( '  - added' )

    print('\n== moneyflow daily: OK\n')

# 获得股票日交易数据 - OK
def sync_tran_daily(conn, period=1):    
    now = datetime.datetime.now()
    # period = 1501
    for n in range(-period+1, 1):
        delta = datetime.timedelta(days=n)
        new_day = now + delta
        trade_date = new_day.strftime('%Y%m%d')
        print( trade_date, end="" )  

        # existDate = False
        sql="SELECT * FROM tran_daily where trade_date='" + trade_date+ "' LIMIT 1"
        df=basic.read_data(conn, sql)
        if len(df) > 0:
            print( '  - existed' )
        else:
            # read data
            # df = pro.daily(trade_date=trade_date)
            df = ts.pro_bar(adj='qfq', start_date=trade_date, end_date=trade_date)
            df.rename( columns={'vol':'volume','pct_chg':'pct_change', 'amount':'value'}, inplace=True )
            print( '  - read: ', df.shape, end="")
            if len(df) > 0:  
                # print( '  - none ')
                # 删除已有数据
                # sql = "delete from tran_daily where trade_date="+trade_date
                # print( 'trade_date :', trade_date )
                # print( 'sql: ', sql )
                # print( 'delete res: ', basic.exec_sql(conn, sql) )
                # 添加数据
                write_no = basic.write_data(conn, df, 'tran_daily', False,'append') 
                if write_no > 0:
                    print( '  - Added', str(write_no))
                else:
                    print( '  - Added 0')
            else:    
                print( '  - none ' )
    print('\n== tran_daily: OK\n')
